In [1]:
import os
import numpy as np
from scipy import integrate
from astropy.modeling.blackbody import blackbody_lambda
from astropy import units as u

In [2]:
# Ratio of continuum energy to bolometric energy, per Osten & Wolk (2015).
Econt_Ebol = 0.6

In [3]:
# Calculate continuum emission of 10,000 K blackbody.
cont = np.arange(1400, 10000, 1) * u.AA
bb_cont = blackbody_lambda(in_x=cont, temperature=10000 * u.Kelvin)

In [4]:
# This handy little function will read in a filter profile file from the SVO Filter Service so we can get the
# approximate min. and max. wavelength of a filter.
def read_svo_file(ifile, min_area=0., min_trans=0., is_transmission=False):
    """
    If the SVO filter contains effective area in cm^2, just specify the
    optional 'min_area' parameter to ignore those below a certain threshold.
    If the SVO filter file contains transmission percentage, set the
    'is_transmission' flag to True and the optional 'min_trans' to ignore
    those wavelengths below that threshold instead.
    NOTE: If transmissions, it assumes they are stored as fractions (i.e.,
    0.0 <= trans <= 1.0).
    """
    with open(ifile, 'r') as inputf:
        flines = inputf.readlines()
    # Parse to get first wavelength with non-zero transmission in the file.
    start_wl = None
    index = 0
    while start_wl is None:
        if index < len(flines):
            if is_transmission:
                if float(flines[index].strip().split()[1]) > min_trans:
                    start_wl = float(flines[index].strip().split()[0])
            else:
                if float(flines[index].strip().split()[1]) > min_area:
                    start_wl = float(flines[index].strip().split()[0])
        else:
            break
        index += 1
    # Parse to get last wavelength with non-zero transmission in the file.
    end_wl = None
    index = -1
    while end_wl is None:
        if index > -1*len(flines):
            if is_transmission:
                if float(flines[index].strip().split()[1]) > min_trans:
                    end_wl = float(flines[index].strip().split()[0])
            else:
                if float(flines[index].strip().split()[1]) > min_area:
                    end_wl = float(flines[index].strip().split()[0])
        else:
            break
        index -= 1
    return (start_wl, end_wl)

In [5]:
# Reproduce the NUV calculation from Brasseur et al. (2019) of the fraction of the bolometric luminosity contained
# within the GALEX NUV band.  Instead of using the filter wavelength range cited in their paper, we rely on the SVO
# filter service and consider the start and end wavelengths to be those that have an effective area of least 5 cm^2
# (we note the results are essentially identical and well within the uncertainties in the approximate calculation).
nuv_filter_file = "GALEX_GALEX.NUV.dat.txt"
if os.path.isfile(nuv_filter_file):
    start_wl, end_wl = read_svo_file(nuv_filter_file, min_area=5)
else:
    raise IOError("Missing SVO lookup file: " + nuv_filter_file + " from repository.")
nuv = np.arange(start_wl, end_wl, 1) * u.AA
bb_nuv = blackbody_lambda(in_x=nuv, temperature=10000 * u.Kelvin)
ENUV_Econt = np.trapz(x=nuv, y=bb_nuv) / np.trapz(x=cont, y=bb_cont)
print('p_bol (NUV) = ', float(ENUV_Econt) * Econt_Ebol)
print('NOTE: p_bol (NUV) using 1771 and 2831 Angstroms as start/stop like in Brasseur et al. 2019:'
          ' 0.13152561432613658')

p_bol (NUV) =  0.13331626928915136
NOTE: p_bol (NUV) using 1771 and 2831 Angstroms as start/stop like in Brasseur et al. 2019: 0.13152561432613658


In [6]:
# Compute the fraction of the bolometric luminosity contained within the GALEX FUV band.
fuv_filter_file = "GALEX_GALEX.FUV.dat.txt"
if os.path.isfile(fuv_filter_file):
    start_wl, end_wl = read_svo_file(fuv_filter_file, min_area=5)
else:
    raise IOError("Missing SVO lookup file: " + fuv_filter_file + " from repository.")
fuv = np.arange(start_wl, end_wl, 1) * u.AA
bb_fuv = blackbody_lambda(in_x=fuv, temperature=10000 * u.Kelvin)
EFUV_Econt = np.trapz(x=fuv, y=bb_fuv) / np.trapz(x=cont, y=bb_cont)
print('p_bol (FUV) = ', float(EFUV_Econt) * Econt_Ebol)

p_bol (FUV) =  0.020583537488146248


Compute the p_bol fraction for Johnson U-band, Kepler, and TESS bandpasses.

In [7]:
# Johnson U-band.
uj_filter_file = "Generic_Johnson.U.dat.txt"
if os.path.isfile(uj_filter_file):
    start_wl, end_wl = read_svo_file(uj_filter_file, is_transmission=True,
                                        min_trans=0.05)
else:
    raise IOError("Missing SVO lookup file: " + uj_filter_file + " from repository.")
uj = np.arange(start_wl, end_wl, 1) * u.AA
bb_uj = blackbody_lambda(in_x=uj, temperature=10000 * u.Kelvin)
EUJ_Econt = np.trapz(x=uj, y=bb_uj) / np.trapz(x=cont, y=bb_cont)
print('p_bol (Johnson U) = ', float(EUJ_Econt) * Econt_Ebol)

p_bol (Johnson U) =  0.11018768905750259


In [8]:
# Kepler band.
kepler_filter_file = "Kepler_Kepler.K.dat.txt"
if os.path.isfile(kepler_filter_file):
    start_wl, end_wl = read_svo_file(kepler_filter_file,
                                        is_transmission=True,
                                        min_trans=0.05)
else:
    raise IOError("Missing SVO lookup file: " + kepler_filter_file + " from repository.")
kep = np.arange(start_wl, end_wl, 1) * u.AA
bb_kep = blackbody_lambda(in_x=kep, temperature=10000 * u.Kelvin)
EKEP_Econt = np.trapz(x=kep, y=bb_kep) / np.trapz(x=cont, y=bb_cont)
print('p_bol (Kepler) = ', float(EKEP_Econt) * Econt_Ebol)

p_bol (Kepler) =  0.24104630049720632


In [9]:
# TESS band.
tess_filter_file = "TESS_TESS.Red.dat.txt"
if os.path.isfile(tess_filter_file):
    start_wl, end_wl = read_svo_file(tess_filter_file,
                                        is_transmission=True,
                                        min_trans=0.05)
else:
    raise IOError("Missing SVO lookup file: " + tess_filter_file + " from repository.")
tess = np.arange(start_wl, end_wl, 1) * u.AA
bb_tess = blackbody_lambda(in_x=tess, temperature=10000 * u.Kelvin)
ETESS_Econt = np.trapz(x=tess, y=bb_tess) / np.trapz(x=cont, y=bb_cont)
print('p_bol (TESS) = ', float(ETESS_Econt) * Econt_Ebol)

p_bol (TESS) =  0.13555505263965456


The bolometric fraction of the Evryscope g' filter is stated directly in Howard et al. 2019 following the same technique, and is p_bol(evryscope) = 0.19.

In [11]:
print('p_bol (Evryscope) = ', 0.19)

p_bol (Evryscope) =  0.19


In [15]:
# Evryscope g' band.
gp_filter_file = "SLOAN_SDSS.gprime_filter.dat.txt"
if os.path.isfile(gp_filter_file):
    start_wl, end_wl = read_svo_file(gp_filter_file,
                                        is_transmission=True,
                                        min_trans=0.05)
else:
    raise IOError("Missing SVO lookup file: " + gp_filter_file + " from repository.")
gp = np.arange(start_wl, end_wl, 1) * u.AA
bb_gp = blackbody_lambda(in_x=gp, temperature=10000 * u.Kelvin)
EGP_Econt = np.trapz(x=gp, y=bb_gp) / np.trapz(x=cont, y=bb_cont)
print('p_bol (Evryscope, ignoring <5% transmission) = ', float(EGP_Econt) * Econt_Ebol)

p_bol (Evryscope, ignoring <5% transmission) =  0.15888794683609475


We differ slightly because of our choice not to include wavelengths that have less than 5% transmission.  Indeed, if you change the 'min_trans' value to 0.0 instead of 0.05 above, we also get 0.19 as reported in Howard et al. (2019).